In [1]:
import xlwings as xw
import pandas as pd
import unicodedata

# Conecta ao Excel já aberto
wb = xw.books['8.0_Relatorio_fechamento junho_2025_VERSÃO FINAL 24-07-2025.xlsm']

In [14]:
def normaliza(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(s))
        if unicodedata.category(c) != 'Mn'
    ).lower()

def achar_coluna_nome(df, texto):
    alvo = normaliza(texto)
    for c in df.columns:
        if alvo in normaliza(str(c)):
            return c
    raise Exception(f'Coluna contendo \"{texto}\" não encontrada.')

def carrega_range(ref):
    if "!" in ref:
        planilha, intervalo = ref.split("!")
        planilha = planilha.strip("'")
        rng = wb.sheets[planilha].range(intervalo)
    else:
        rng = wb.names[ref].refers_to_range
    data = rng.options(pd.DataFrame, header=3, index=False).value
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[-1] for col in data.columns]
    col_data      = achar_coluna_nome(data, 'data')
    col_asset     = achar_coluna_nome(data, 'patrimônio')
    col_pnl_perc  = achar_coluna_nome(data, 'Dia')
    try:
        col_pnl_fin   = achar_coluna_nome(data, 'PnL Dia')
    except Exception:
        col_pnl_fin   = achar_coluna_nome(data, '1D')
    cols = [col_data, col_asset, col_pnl_perc, col_pnl_fin]
    data = data[cols]
    data.columns = ['overview_date', 'asset_value', 'dtd_ativo_pct', 'dtd_ativo_fin']
    data['overview_date'] = pd.to_datetime(data['overview_date'])
    return data

In [ ]:
# ----------- LS -----------
ranges_nomeados = ['caixa_cart_2', 'risco_cart_2', 'operacao_hypera_2',"'BRT RV 3ºs'!GM:GV","'BRT Multi Próprio'!CB:CK",
                   "'BRT Multi 3ºs'!JD:JM","'BRT Crédito'!OV:PE","'BRT Ilíquido'!IX:JG"]
book_names = ['Caixa >> NC Cash Perenne', 'Risco', 'Risco >> HYPE','Risco >> Renda Variável Terceiros','Risco >> Multimercado Próprio','Risco >> Multimercado Terceiros',
              'Risco >> Crédito','Risco >> Ilíquidos']
portfolio_id = 1296

In [17]:
# ----------- MS -----------

ranges_nomeados = ["caixa_cart_3","'BRT Risco Ex-FIP'!AJ:AS","'OPERAÇÃO HYPERA'!AJ:AS","'BRT RV 3ºs'!JY:KH","'BRT Multi Próprio'!DI:DR",
"'BRT Multi 3ºs'!MY:NH","'BRT Crédito'!TM:TV","'BRT Ilíquido'!MH:MQ","'BRT Outros'!BG:BP"]
book_names = ['Caixa >> NC Cash Perenne', 'Risco', 'Risco >> HYPE','Risco >> Renda Variável Terceiros','Risco >> Multimercado Próprio','Risco >> Multimercado Terceiros',
              'Risco >> Crédito','Risco >> Ilíquidos','Risco >> Outros']
portfolio_id = 1297


In [18]:
# ----------- USO -----------
inicio = '2024-12-31'
fim = '2025-06-30'

dfs = []
for nome, book in zip(ranges_nomeados, book_names):
    df = carrega_range(nome)
    df = df[(df['overview_date'] >= inicio) & (df['overview_date'] <= fim)]
    df['portfolio_id'] = portfolio_id
    df['book_name'] = book
    df['instrument_name'] = "OUTROS"
    df['exposure_value'] = df['asset_value']
    # Esses shifts só fazem sentido dentro do mesmo book! (aqui é seguro)
    df['asset_value_ontem'] = df['asset_value'].shift(1)
    df['exposure_value_ontem'] = df['exposure_value'].shift(1)
    df = df[['overview_date', 'portfolio_id', 'book_name', 'instrument_name', 'asset_value',
             'dtd_ativo_pct', 'dtd_ativo_fin', 'exposure_value',
             'asset_value_ontem', 'exposure_value_ontem']]
    dfs.append(df)

df_out = pd.concat(dfs, ignore_index=True).sort_values(['book_name', 'overview_date']).reset_index(drop=True)


In [19]:
# --- AJUSTE BOOK "RISCO" e TODAS AS COLUNAS RELACIONADAS ---
mascara_risco = df_out['book_name'].str.lower().str.startswith('risco')
mask_exato_risco = df_out['book_name'].str.lower().str.strip() == 'risco'

for dia in df_out['overview_date'].unique():
    idxs_risco = df_out.index[(df_out['overview_date'] == dia) & mask_exato_risco]
    idxs_risco_outros = df_out.index[(df_out['overview_date'] == dia) & mascara_risco & ~mask_exato_risco]
    if len(idxs_risco) == 1:
        idx_risco = idxs_risco[0]
        # Soma dos outros riscos do dia
        soma_asset_outros = df_out.loc[idxs_risco_outros, 'asset_value'].sum()
        soma_ativo_outros = df_out.loc[idxs_risco_outros, 'dtd_ativo_fin'].sum()
        soma_exposure_outros = df_out.loc[idxs_risco_outros, 'exposure_value'].sum()
        soma_asset_ontem_outros = df_out.loc[idxs_risco_outros, 'asset_value_ontem'].sum()
        soma_exposure_ontem_outros = df_out.loc[idxs_risco_outros, 'exposure_value_ontem'].sum()
        
        # Valor atual de "Risco"
        asset_risco = df_out.at[idx_risco, 'asset_value']
        ativo_risco = df_out.at[idx_risco, 'dtd_ativo_fin']
        exposure_risco = df_out.at[idx_risco, 'exposure_value']
        asset_ontem_risco = df_out.at[idx_risco, 'asset_value_ontem']
        exposure_ontem_risco = df_out.at[idx_risco, 'exposure_value_ontem']

        # Ajusta todos os campos numéricos
        novo_asset = asset_risco - soma_asset_outros
        novo_ativo = ativo_risco - soma_ativo_outros
        novo_exposure = exposure_risco - soma_exposure_outros
        novo_asset_ontem = (asset_ontem_risco or 0) - soma_asset_ontem_outros
        novo_exposure_ontem = (exposure_ontem_risco or 0) - soma_exposure_ontem_outros
        novo_pct = novo_ativo / novo_asset if novo_asset != 0 else 0

        # Aplica de volta
        df_out.at[idx_risco, 'asset_value'] = novo_asset
        df_out.at[idx_risco, 'dtd_ativo_fin'] = novo_ativo
        df_out.at[idx_risco, 'exposure_value'] = novo_exposure
        df_out.at[idx_risco, 'asset_value_ontem'] = novo_asset_ontem
        df_out.at[idx_risco, 'exposure_value_ontem'] = novo_exposure_ontem
        df_out.at[idx_risco, 'dtd_ativo_pct'] = novo_pct
        df_out.at[idx_risco, 'book_name'] = 'Risco >> Caixas e Provisionamentos'
    elif len(idxs_risco) > 1:
        print(f"Mais de um 'Risco' encontrado em {dia.date()}! Cheque seu input.")

df_out = df_out.sort_values(['book_name', 'overview_date']).reset_index(drop=True)
display(df_out)
df_out.to_clipboard(index=False)

,overview_date,portfolio_id,book_name,instrument_name,asset_value,dtd_ativo_pct,dtd_ativo_fin,exposure_value,asset_value_ontem,exposure_value_ontem
0,2024-12-31,1297,Caixa >> NC Cash Perenne,OUTROS,2.151491e+08,0.000468,100563.38,2.151491e+08,NaN,NaN
1,2025-01-02,1297,Caixa >> NC Cash Perenne,OUTROS,2.152507e+08,0.000473,101660.46,2.152507e+08,2.151491e+08,2.151491e+08
2,2025-01-03,1297,Caixa >> NC Cash Perenne,OUTROS,2.153598e+08,0.000507,109039.52,2.153598e+08,2.152507e+08,2.152507e+08
3,2025-01-06,1297,Caixa >> NC Cash Perenne,OUTROS,2.154419e+08,0.000382,82186.15,2.154419e+08,2.153598e+08,2.153598e+08
4,2025-01-07,1297,Caixa >> NC Cash Perenne,OUTROS,2.230654e+08,0.000573,123419.66,2.230654e+08,2.154419e+08,2.154419e+08
...,...,...,...,...,...,...,...,...,...,...
1101,2025-06-24,1297,Risco >> Renda Variável Terceiros,OUTROS,6.219526e+06,0.011119,68392.10,6.219526e+06,6.151134e+06,6.151134e+06
1102,2025-06-25,1297,Risco >> Renda Variável Terceiros,OUTROS,6.170828e+06,-0.007830,-48697.93,6.170828e+06,6.219526e+06,6.219526e+06
1103,2025-06-26,1297,Risco >> Renda Variável Terceiros,OUTROS,6.220311e+06,0.008019,49482.67,6.220311e+06,6.170828e+06,6.170828e+06
1104,2025-06-27,1297,Risco >> Renda Variável Terceiros,OUTROS,6.202196e+06,-0.002912,-18114.56,6.202196e+06,6.220311e+06,6.220311e+06
